In [1]:
import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from args_reader import read_args
from scicrunch_client_fast_mod import scicrunch
import sys
import time
from interlex_sql import interlex_sql
import re
from collections import defaultdict
args = read_args(keys='../keys.txt', production='-p')
sql = interlex_sql(engine_key=args['engine_key'])
sci = scicrunch(key=args['api_key'], base_path=args['base_path'], engine_key=args['engine_key'])

## placticity

In [6]:
#file_used_to_check = '/Users/tmsincomb/Desktop/NDA/nda-cdes-second-tier-only-update-ready.json'
file_used_to_check = '/Users/tmsincomb/Desktop/NDA/nda-cdes-first-tier-only-update-ready.json'
#file_used_to_check = '/Users/tmsincomb/Desktop/NDA/nda-cdes-second-tier-annotations.json'
raw_data = json.load(open(file_used_to_check, 'r'))

In [34]:
raw_data[0]

{'annotation_tid': 15074, 'tid': '230847', 'value': 'condition : None'}

In [40]:
print(list(raw_data)[0])
print(raw_data[list(raw_data)[0]])

12-Item Grit Scale
{'label': '12-Item Grit Scale', 'synonyms': 'grit01', 'definition': ' ', 'type': 'cde', 'annotation_tid': 15074, 'value': ['sources : RDoC', 'categories : Questionnaire', 'dataType : Clinical Assessments', 'status : Published', 'modifiedDate : 1515012631513'], 'superclass_label': 'NIMH Data Archive Common Data Element', 'superclass_ilx': 'ilx_0115072', 'superclass_tid': 15075}


In [42]:
raw_data['Ticarcillin']

KeyError: 'Ticarcillin'

In [32]:
labels_to_ids = sql.get_labels_to_ids_dict()

In [2]:
annotation_df = sql.get_annotations()

In [4]:
annotation_df

,item_id,label,ilx,anno_tid,anno_label,anno_ilx,tid,annotation_tid,value
0,182,Abnormal Involuntary Movement Scale,ilx_0100181,15074,hasConstraint,ilx_0115071,182,15074,categories : Questionnaire
1,182,Abnormal Involuntary Movement Scale,ilx_0100181,15074,hasConstraint,ilx_0115071,182,15074,status : Published
2,182,Abnormal Involuntary Movement Scale,ilx_0100181,15074,hasConstraint,ilx_0115071,182,15074,dataType : Clinical Assessments
3,182,Abnormal Involuntary Movement Scale,ilx_0100181,15074,hasConstraint,ilx_0115071,182,15074,sources : NDAR; NDCT; RDoC
4,182,Abnormal Involuntary Movement Scale,ilx_0100181,15074,hasConstraint,ilx_0115071,182,15074,modifiedDate : 1515012631513
5,360,Adult dorsomedial neurosecretory cell,ilx_0100359,12770,Overlaps some,ilx_0112769,360,12770,adult tritocerebrum
6,360,Adult dorsomedial neurosecretory cell,ilx_0100359,12770,Overlaps some,ilx_0112769,360,12770,superior lateral protocerebrum
7,360,Adult dorsomedial neurosecretory cell,ilx_0100359,12770,Overlaps some,ilx_0112769,360,12770,hypocerebral ganglion
8,559,Amoxicillin,ilx_0100558,15024,Source,ilx_0115023,559,15024,VISION-SCI
9,559,Amoxicillin,ilx_0100558,15025,Variable type,ilx_0115024,559,15025,Surgery


In [6]:
annotation_dict = annotation_df.to_dict('records')

In [40]:
annotation_df.loc[annotation_df.label == "Benton Facial Recognition Test"]

,item_id,label,ilx,anno_tid,anno_label,anno_ilx,tid,annotation_tid,value


In [41]:
len(raw_data.keys())

2234

# checking annotations

In [46]:
%%time
reference = defaultdict(list)
for memo in annotation_dict:
    reference[memo['label'].lower()].append(memo)

hit = set()
for label in raw_data.keys():
    if label != 'Benton Facial Recognition Test': continue
    label = label.replace("'",'&#39;').lower()
    if reference.get(label): 
        if not reference[label]:
            hit.add(label)
        print(reference[label])
        for annotation in reference[label]:
            if annotation['anno_label'] != 'hasConstraint':
                hit.add(label)
    else:
        hit.add(label)
print('Num of broken cdes:',len(hit))

[{'item_id': 1216, 'label': 'Benton facial recognition test', 'ilx': 'ilx_0101215', 'anno_tid': 15074, 'anno_label': 'hasConstraint', 'anno_ilx': 'ilx_0115071', 'tid': 1216, 'annotation_tid': 15074, 'value': 'modifiedDate : 1515012631513'}, {'item_id': 1216, 'label': 'Benton facial recognition test', 'ilx': 'ilx_0101215', 'anno_tid': 15074, 'anno_label': 'hasConstraint', 'anno_ilx': 'ilx_0115071', 'tid': 1216, 'annotation_tid': 15074, 'value': 'dataType : Clinical Assessments'}, {'item_id': 1216, 'label': 'Benton facial recognition test', 'ilx': 'ilx_0101215', 'anno_tid': 15074, 'anno_label': 'hasConstraint', 'anno_ilx': 'ilx_0115071', 'tid': 1216, 'annotation_tid': 15074, 'value': 'categories : Cognitive'}, {'item_id': 1216, 'label': 'Benton facial recognition test', 'ilx': 'ilx_0101215', 'anno_tid': 15074, 'anno_label': 'hasConstraint', 'anno_ilx': 'ilx_0115071', 'tid': 1216, 'annotation_tid': 15074, 'value': 'sources : NDAR'}, {'item_id': 1216, 'label': 'Benton facial recognition te

In [49]:
broken_terms = []
broken_annotations = []
for h in hit:
    tid = labels_to_ids(raw_data[h]['label'])
    annotation_tid = labels_to_ids(raw_data[h]['annotation_tid'])
    for value in raw_data[h]['value']: 
        broken_annotations.appen({
            'tid':tid,
            'annotation_tid': annotation_tid,
            'value':value,
        })
    broken_terms.append({
        'label':raw_data[h]['label'],
        'definition':raw_data[h]['definition']
    })

In [4]:
terms_df = sql.get_terms()

In [2]:
label_to_ilx = sql.get_labels_to_ilx_dict()
label_to_id = sql.get_labels_to_ids_dict()

In [62]:
id_to_label = {v:k for k, v in label_to_id.items()}

In [3]:
superclasses_df = sql.get_superclasses()

In [8]:
id_to_super_tid = {int(v['tid']):int(v['id']) for v in superclasses_df.to_dict('records')}

In [17]:
cde_labels = {l.lower():True for l in list(raw_data)}
for i, row in terms_df.iterrows():
    tid = int(row.id)
    if tid < 150000 or tid > 304300: continue
    if 'size : ' in row.definition.lower():#and cde_labels.get(row.label.lower()):
        print(tid, row.label, row.definition)
        break

150000 mini_k_dep_life (Mini International Neuropsychiatric Interview. Part I) description : alcohol dependence (lifetime); condition : None; filterElement : 1; notes : 0=no; 1=yes; 99 =missing; required : Recommended; size : None; type : Integer; valueRange : 0;1; 99


In [52]:
labels = sorted([l.lower() for l in list(terms_df.label)])
visited = {}
doubles = {}
for label in labels:
    if not visited.get(label):
        visited[label] = True
    else:
        doubles[label] = True

In [53]:
hits = defaultdict(dict)
for i, row in terms_df.iterrows():
    if row['label'].lower() == 'treatment':
        print(row)
        print()
    if doubles.get(row['label'].lower()):
        if row['type'] == 'term':
            hits[row['label'].lower()].update({'term':True})
        if row['type'] == 'cde':
            hits[row['label'].lower()].update({'cde':True})

id                                                                11926
ilx                                                         ilx_0111925
orig_uid                                                          32290
orig_cid                                                              0
uid                                                               32290
cid                                                                   0
label                                                         Treatment
definition            A processual entity whose completion is hypoth...
comment                                                                
type                                                               term
version                                                               2
status                                                                0
display_superclass                                                    1
orig_time                                                    148

In [54]:
double_labels = []
for k, hit in hits.items():
    if hit.get('term') and hit.get('cde'):
        double_labels.append(k)
#with open('/Users/tmsincomb/Desktop/double_labels.txt', 'w') as outfile:
#    outfile.write('\n'.join(double_labels))

In [55]:
double_ilx = {}
for l in double_labels:
    ilx = label_to_ilx[l]
    double_ilx['http://uri.interlex.org/base/'+ilx] = l

In [20]:
import rdflib
g_ilx = rdflib.Graph()
g_ilx.parse('/Users/tmsincomb/Desktop/interlexutils/Interlex.ttl', format='turtle')

<Graph identifier=N63a2a06ec4b845618b5d29bf05ec023d (<class 'rdflib.graph.Graph'>)>

In [57]:
terms_def_to_fix = []
for triple in g_ilx.triples((None, None, None)):
    if str(triple[1]) == "http://purl.obolibrary.org/obo/IAO_0000115":
        if double_ilx.get(str(triple[0])):
            terms_def_to_fix.append({
            'id':label_to_id[double_ilx.get(str(triple[0]))],
            #'label':double_ilx.get(str(triple[0])),
            'definition':str(triple[2])
            })
            
with open('/Users/tmsincomb/Desktop/terms_def_to_fix.json', 'w') as f:
    json.dump(terms_def_to_fix, f, indent=4)

In [6]:
%%time
hits = []
for i, label in enumerate(list(doubles)[:]):
    if i % 100 == 0: print(i)
    #if label != 'benton facial recognition test': continue
    sub_df = terms_df.loc[terms_df.label.str.lower() == label]
    if not sub_df.loc[sub_df.type.str.contains('cde')].empty:
        if not sub_df.loc[sub_df.type.str.contains('term')].empty:
            hits.append(label)
                    
with open('/Users/tmsincomb/Desktop/double_labels.txt', 'w') as outfile:
    outfile.write('\n'.join(hits))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900


KeyboardInterrupt: 

In [81]:
15*300/60.0

75.0

In [79]:
len(hits)

1

In [19]:
%%time
not_passed = set()
for label in list(raw_data.keys())[:1]:
    sub_df = annotation_df.loc[annotation_df.label == label]
    #for anno_label in sub_df.anno_label:
    #    if anno_label != 'hasConstraint':
    #        print(anno_label)
    #        not_passed.add(label)
not_passed

CPU times: user 189 ms, sys: 2.64 ms, total: 191 ms
Wall time: 190 ms
